In [1]:
import pandas as pd
import numpy as np

import os

In [13]:
df_li_1 = []
for location in ['5.In_front_of_engineering',
 '10.Gate-5 warehouse truck loading',
 '6.Engineering_Store',
 '1.Gate-1_Sawing_Machine_Area',
 '9.Stacker_bay_devi_stack_counter',
 '8.In_front_of_engineering_and_canteen']:
    df_li_1.append(pd.read_csv('../../../dataset/helmet_n_vest/helmet_vest_8_04_2019/'+location+'/_retinanet.csv',header=None))
    

In [22]:
df_li_2 = []
for f in ['1-100', '501-600', '301-400', '101-200', '401-500', '701-800']:
    df_li_2.append(pd.read_csv('../../../dataset/helmet_n_vest/hot_mill_view_low_light_26_04_2019/Train_Frame/'+f+'/_retinanet.csv',header=None))
    

In [32]:
# # changing filepath
# csv_file_path = '../../../dataset/helmet_n_vest/pallavi_data_k80_1/annotations_by_pallavi/cam02/_retinanet.csv'

# df4 = pd.read_csv(csv_file_path,header=None)

# file_path = []
# for i in range(df1.shape[0]):
#     file_path.append(os.path.abspath('../../../dataset/helmet_n_vest/pallavi_data_k80_1/cam02/'+os.path.basename(df1.iloc[i][0])))

# df4[0] = file_path

# df4.to_csv(csv_file_path,index=False,header=False)

In [78]:
df1 = pd.read_csv('../../../dataset/helmet_n_vest/pallavi_data_k80_1/annotations_by_pallavi/cam02/_retinanet.csv',header=None)
df2 = pd.read_csv('../../../dataset/helmet_n_vest/pallavi_data_k80_1/annotations_by_pallavi/Annealing View_Annotation/_retinanet.csv',header=None)
df3 = pd.read_csv('../../../dataset/helmet_n_vest/pallavi_data_k80_1/annotations_by_pallavi/Hot_Mill_View_Annotation/_retinanet.csv',header=None)
df4 = pd.read_csv('../../../dataset/helmet_n_vest/annealing_view_low_light_26_04_2019/101-200/_retinanet.csv',header=None)

In [79]:
# df2 = pd.read_csv('Hot_Mill_View_Annotation/_retinanet.csv',header=None)
# df3 = pd.read_csv('Annealing View_Annotation/_retinanet.csv',header=None)
# df1 = pd.read_csv('../../../dataset/helmet_n_vest/pallavi_data_k80_1/annotations_by_pallavi/google/_retinanet.csv',header=None)
# df4 = pd.read_csv('../../../dataset/helmet_n_vest/pallavi_data_k80_1/annotations_by_pallavi/cam02/_retinanet.csv',header=None)


# df2[0]  = df2[0].apply(lambda x : x.replace('jpeg','jpg'))
# df3[0]  = df3[0].apply(lambda x : x.replace('jpeg','jpg'))
# df4[0]  = df4[0].apply(lambda x : x.replace('jpeg','jpg'))

df_total = pd.concat([df1,df2,df3,df4]+df_li_1+df_li_2)

In [80]:
import pandas_ml as pdml

In [81]:
df_total.reset_index(drop=True,inplace=True)

In [82]:
df_total[5].value_counts()

person       11760
helmet        8452
worker        5197
vest          3959
no_helmet     3493
no_vest       1480
Name: 5, dtype: int64

In [83]:

#df_total.loc[df_total[df_total[5]=='worker'].index,5] = 'no_vest'

#df_total[5].value_counts()


In [84]:
df_total.columns = [0, 1, 2, 3, 4, 'target']

#### oversampling

In [85]:
df_total.shape

(34341, 6)

In [86]:
df_total = df_total[df_total[1]>0]
df_total = df_total[df_total[2]>0]
df_total = df_total[df_total[3]>0]
df_total = df_total[df_total[4]>0]

In [87]:
df_total.shape

(33938, 6)

In [88]:
df_pdml = pdml.ModelFrame(df_total,target='target')

sampler = df_pdml.imbalance.over_sampling.RandomOverSampler()

#sampler

sampled = df_pdml.fit_sample(sampler)

In [89]:
sampled.target.value_counts()

no_helmet    11536
worker       11536
no_vest      11536
vest         11536
helmet       11536
person       11536
Name: target, dtype: int64

In [90]:
sampled[[0, 1, 2, 3, 4, 'target']].to_csv('../../../dataset/helmet_n_vest/combined_annos_retinanet_1.csv',header=None,index=False)

In [32]:
from sklearn.model_selection import StratifiedShuffleSplit
X = sampled.data.values
y = sampled.target.values
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)

print(sss)       

for train_index, test_index in sss.split(X, y):
   #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]



StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.2,
            train_size=None)


In [33]:
sampled.loc[train_index][[0, 1, 2, 3, 4, 'target']].to_csv('combined_annos_retinanet_train.csv',header=None,index=False)
sampled.loc[test_index][[0, 1, 2, 3, 4, 'target']].to_csv('combined_annos_retinanet_test.csv',header=None,index=False)

In [15]:
sampled.loc[train_index].iloc[1592]

target                                            no_helmet
0         /mnt/external_drive/Notebooks/helmet_detection...
1                                                      1360
2                                                       283
3                                                      1397
4                                                       317
Name: 19230, dtype: object